In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
import cv2
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.optimizers import Adam
from keras_unet_collection import models
from PIL import Image
plt.style.use("ggplot")
%matplotlib inline

## DL Swin without k-fold

In [ ]:
#define directory where images and masks are located on local disk
image_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_images_RF/insert_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'
#'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'

#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom functions
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))


#define some hyperparameters
num_labels = 2  #Binary classificaion (missmatch with literature/code examples!)
batch_size = 2  #keep it smaller than 3
epochs = 2


#normalize images
image_dataset = np.array(image_dataset)/255
#do not normalize masks, just rescale to 0 to 1. Add RGB-Chanel (3) to mask.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255
mask_dataset = tf.keras.utils.to_categorical(mask_dataset)

""" plt.imshow(mask_dataset[1])
plt.show()
plt.imshow(image_dataset[1])
plt.show() """

print(image_dataset[1].shape)
print(mask_dataset[1].shape)

#define callback function
callbacks = [
  EarlyStopping(patience=8, verbose=1),
  ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
  ModelCheckpoint('A_modelSwinV1.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
  CSVLogger('A_modelSwinV1.csv', separator=',', append=False)]
    
#define the model architecture
#this model requires depth >= 2
model = models.swin_unet_2d((256, 256, 3), filter_num_begin=64, n_labels=num_labels, depth=4, stack_num_down=2, stack_num_up=2, 
                            patch_size=(4, 4), num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=512, 
                            output_activation='Softmax', shift_window=True, name='swin_unet') #Guess: Shift_window = False

#compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
            metrics=['accuracy', IoU])

#split dataset into training and validation set
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

#fit model to data
resunet_history = model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(X_test, y_test), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

## Working

In [2]:
#define directory where images and masks are located on local disk
image_directory = 'D:/UniBas/Bachelorarbeit/some_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/some_masks/'
#'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'

#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom functions
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))


#define some hyperparameters
num_labels = 2  #Binary classificaion (missmatch with literature/code examples!)
batch_size = 2  #keep it smaller than 3
epochs = 2

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

#normalize images
image_dataset = np.array(image_dataset)/255
#do not normalize masks, just rescale to 0 to 1. Add RGB-Chanel (3) to mask.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255
mask_dataset = tf.keras.utils.to_categorical(mask_dataset)

fold_no = 1
for train, test in kfold.split(image_dataset, mask_dataset):
#define callback function
    callbacks = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint(f'B_Kfold{fold_no}_SwinV1.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
    CSVLogger(f'B_Kfold{fold_no}_SwinV1.csv', separator=',', append=False)]
        
    #define the model architecture
    #this model requires depth >= 2
    model = models.swin_unet_2d((256, 256, 3), filter_num_begin=64, n_labels=num_labels, depth=4, stack_num_down=2, stack_num_up=2, 
                                patch_size=(4, 4), num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=512, 
                                output_activation='Softmax', shift_window=True, name='swin_unet') #Guess: Shift_window = False

    #compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
                metrics=['accuracy', IoU])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    #fit model to data
    resunet_history = model.fit(image_dataset[train], mask_dataset[train], 
                        verbose=1,
                        batch_size = batch_size,
                        validation_data=(image_dataset[test], mask_dataset[test]), 
                        shuffle=False,
                        epochs=epochs,
                        callbacks=callbacks)
    
    fold_no += 1


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/2
4/4 [==============================] - 21s 2s/step - loss: 0.9634 - accuracy: 0.6230 - IoU: 0.7094 - val_loss: 0.3970 - val_accuracy: 0.9223 - val_IoU: 0.9141

Epoch 00001: val_loss improved from inf to 0.39704, saving model to B_Kfold1_SwinV1.h5


KeyboardInterrupt: 

## DL Swin with k-Fold (not working)

In [12]:
#define directory where images and masks are located on local disk
image_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_images_RF/insert_images/'
mask_directory = 'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'
#'D:/UniBas/Bachelorarbeit/Img_masks/DeepACSA_masks_RF/insert_masks/'

#define the properties and empty list for resized images and masks
SIZE = 256
image_dataset = []
mask_dataset = []


#define custom functions
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou

#enumerate and resize images/masks
images = os.listdir(image_directory)
for i, image_name in enumerate(images):    #enumerate method adds a counter and returns the enumerate object
    if (image_name.split('.')[1] == 'tif'):
        #print(image_directory+image_name)
        image = cv2.imread(image_directory+image_name, 1)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))


masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'tif'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

#define some hyperparameters
num_labels = 1  #Binary classificaion (missmatch on coding examples! 1 or 2 for binary classification)
batch_size = 2  #keep it smaller than 3
epochs = 2
num_folds = 5   #define the number of folds (usually 5-10 folds)

#normalize images
image_dataset = np.array(image_dataset)/255
#do not normalize masks, just rescale to 0 to 1. Add RGB-Chanel (3) to mask.
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255
mask_dataset = tf.keras.utils.to_categorical(mask_dataset)

#define K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

#define per-fold score containers 
acc_per_fold = []
loss_per_fold = []
IoU_per_fold = []

#K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(image_dataset, mask_dataset):
  callbacks = [
    EarlyStopping(patience=8, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.00001, verbose=1),
    ModelCheckpoint(f'K-foldno{fold_no}-swinunet-V1-VL-256.h5', verbose=1, save_best_only=True, save_weights_only=False), # Give the model a name (the .h5 part)
    CSVLogger(f'K-foldno{fold_no}-swinunet-V1-VL-256.csv', separator=',', append=False)]
  
  #define the model architecture
  #this model requires depth >= 2
  model = models.swin_unet_2d((256, 256, 3), filter_num_begin=64, n_labels=num_labels, depth=4, stack_num_down=2, stack_num_up=2, 
                            patch_size=(4, 4), num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=512, 
                            output_activation='Softmax', shift_window=False, name='swin_unet') #Guess: Shift_window = False

  #compile the model
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 1e-3), 
              metrics=['accuracy', IoU])

  #generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  #fit model on data
  swin_unet_history = model.fit(image_dataset[train], mask_dataset[train], 
                    verbose=1,
                    batch_size = batch_size,
                    validation_data=(image_dataset[test], mask_dataset[test]), 
                    shuffle=False,
                    epochs=epochs,
                    callbacks=callbacks)

  #append evaluation values for every fold to a list
  acc_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[1])
  loss_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[0])
  IoU_per_fold.append(model.evaluate(image_dataset[test], mask_dataset[test])[2])

  #increase fold number
  fold_no += 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]} - IoU: {IoU_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print(f'> IoU: {np.mean(IoU_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/2
205/205 [==============================] - 293s 1s/step - loss: 7.6246 - accuracy: 0.5000 - IoU: 1.0000 - val_loss: 7.6246 - val_accuracy: 0.5000 - val_IoU: 1.0000

Epoch 00001: val_loss improved from inf to 7.62457, saving model to K-foldno1-swinunet-V1-VL-256.h5
Epoch 2/2
205/205 [==============================] - 255s 1s/step - loss: 7.6246 - accuracy: 0.5000 - IoU: 1.0000 - val_loss: 7.6246 - val_accuracy: 0.5000 - val_IoU: 1.0000

Epoch 00002: val_loss did not improve from 7.62457
4/4 [==============================] - 12s 3s/step - loss: 7.6246 - accuracy: 0.5000 - IoU: 1.0000
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/2
144/205 [====================>.........] - ETA: 1:19 - loss: 7.6246 - accuracy: 0.5000 - IoU: 1.0000